### *Module Loading*

In [ ]:
import sys
import faiss
from subprocess import PIPE, run
from IPython.display import display as ip_display

### *External Module Loading*

In [ ]:
external_modules_path = '..\\nn_likelihood_modules'
sys.path.append(external_modules_path)

In [ ]:
from basic_network_structure import *
from common_imports import *
from common_use_functions import *
from constant import *
from defined_data_structure import *
from defined_network_structure import *
from experim_neural_network import *
from experim_preparation import *
from generate_activation_level import *
from pytorch_model_predict import *
from vector_preprocessing import *
from ResNet import *
from experim_ResNet import *
from cifar_10_data_prep import *
from pytorch_swintransformer_modified import *
from tiny_imagenet_data_prep import *
from sensitivity_analysis import *
from deep_KNN import *

### *GPU verification*

In [ ]:
# Get the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nb_gpu = torch.cuda.device_count()
if nb_gpu > 0:
    print(torch.cuda.get_device_name(0))
else:
    print("CPU")

### *Working directory*

In [ ]:
# Current path
current_path = os.path.abspath(os.getcwd())

### *Load configurations and data*

In [ ]:
"""
All the parameters in this part should be configured
"""
# Experience path
experim_path = current_path

# File extensions
json_ext = '.json'
np_ext = '.npy'
csv_ext = '.csv'

# ResNet model prefix
model_name_prefix = 'swin'

# Image max pixel value
image_max_pix_val = 255

# Tested sets name
train_set_name = 'train'
test_set_name = 'test'
valid_set_name = 'valid'
input_extension = 'X'
label_extension = 'Y'

# Save paths
model_save_path = path_join(experim_path, 'experim_models_swin')

"""
The following parameters should be configured according to your experiments
"""

# Trained model name 
trained_net_name = 'swin_b_v2_1024' # You can select any model from the "experim_models_swin" folder

# ResNet related params
net_model_name = 'swin_b_v2_1024'# The model name should be coherent with your chosen model

# Tiny-imagenet datafolder path
tiny_imagenet_path = 'D:\\Doctorat\\research\\Tiny_imagenet_experim\\tiny-imagenet-200-reform\\'
tiny_imagenet_val_path = path_join(tiny_imagenet_path, 'reform_val')
tiny_imagenet_train_path = path_join(tiny_imagenet_path, 'reform_train')

# Tiny-imagenet-c datafolder path
tiny_imagenet_c_path = 'D:\\Doctorat\\research\\Tiny_imagenet_experim\\Tiny-ImageNet-C-reform\\'

# Dataset general informations
data_set_infos = {
    'nb_classes' : 200
}

# Distance decision filtering threshold
std_threshold_coeff = 2

# The method to determining the significant neurons (mean or most_common)
sobol_filter_method = 'mean'

# The end number used for the "top" filtering method (1 means we want the top-1, 2 for top-2 etc.)
top_index_end = 1

# Rscript launch params
Rscript_path = 'C:\\Program Files\\R\\R-4.3.3\\bin\\Rscript.exe'

# The number of considered k-nearst neighbors
k = 200

"""
"""

# The output folder
output_path = path_join(experim_path, 'output')

# Build the class list
class_list = list(range(data_set_infos['nb_classes']))

# Batch size for the dataloader creation
torch_batch_size = 4

In [ ]:
# Create the folders
create_directory(output_path)

### *Experiment preparation*

In [ ]:
# Get the tiny imagenet training set and test set
tiny_imagenet_train_dataset, tiny_imagenet_test_dataset = get_tiny_imagenet_without_transform(tiny_imagenet_train_path, tiny_imagenet_val_path)

In [ ]:
# The column names of the filtering results
column_names_OOD_filtering = ['transformation', 'nb_examples', 'nb_OOD', 'nb_InD', 'total_acc', 'OOD_acc', 'InD_acc']

### *Load the trained Network*

In [ ]:
# Load the network
trained_net = load_model_by_net_name(model_save_path, trained_net_name)

### *Move the model to GPU*

In [ ]:
# Move to gpu
trained_net.cuda()

### *Tiny imagenet dataset preparation*

In [ ]:
# Dataloader building
train_loader = create_loader_from_torch_dataset(tiny_imagenet_train_dataset, batch_size=torch_batch_size, shuffle=False, num_workers=0)
test_loader = create_loader_from_torch_dataset(tiny_imagenet_test_dataset, batch_size=torch_batch_size, shuffle=False, num_workers=0)

### *Evaluate the activation levels for the original training and test sets of Tiny-Imagenet*

In [ ]:
# Get the training set activation levels 
train_actLevels = obtain_activation_levels(trained_net,
                                           train_loader, 'train', with_predict_class=True, loss_type='cross_entropy')

In [ ]:
# Get the test set activation levels 
test_actLevels = obtain_activation_levels(trained_net,
                                           test_loader, 'test', with_predict_class=True, loss_type='cross_entropy')

### *Sobol index evaluation*

In [ ]:
# Last hidden layer Id
last_hidden_layerId = list(train_actLevels['actLevel'].keys())[-1]

In [ ]:
# Get the correctly predicted index
correctly_predicted_bools = train_actLevels['class'] == train_actLevels['predict_class']
# Get the corresponding data
last_hidden_actLevels = train_actLevels['actLevel'][last_hidden_layerId][correctly_predicted_bools.reshape(-1)]

In [ ]:
# Get the last layer parameters
model_params = get_model_parameters(trained_net, to_numpy=True)
final_linear_params = model_params['head_out.0']

In [ ]:
# Normalization of weight and input
# Copy the original activation levels and weights
normalized_last_hidden_actLevel = copy.deepcopy(last_hidden_actLevels)
upped_final_linear_params = copy.deepcopy(final_linear_params)
# Check the min and max values of each neuron
last_hidden_actLevel_max = np.max(last_hidden_actLevels, axis=0)
last_hidden_actLevel_min = np.min(last_hidden_actLevels, axis=0) # Not used, just for verification
# Iterate over the maximum values and normalize the input
for index, neuron_max in enumerate(last_hidden_actLevel_max):
    if neuron_max != 0:
        normalized_last_hidden_actLevel[:,index] = normalized_last_hidden_actLevel[:,index] / neuron_max
        upped_final_linear_params['weight'][:,index] = upped_final_linear_params['weight'][:,index] * neuron_max

In [ ]:
# Build the final linear parameters per class and assign the real data
used_linear_params = upped_final_linear_params
data = normalized_last_hidden_actLevel
# Number of variables
nb_vars = data.shape[1]

In [ ]:
## Build the X and y for the sobol index evaluation in R
# Get the neuron names
neuron_names = ['neuron_'+str(index) for index in range(nb_vars)]
# Build the X dataframe
R_X = pd.DataFrame(data, columns=neuron_names)

In [ ]:
# Build the dataframe that stores weights and bias
R_network_params_weight_columns = [*(['weight_'+str(index) for index in range(used_linear_params['weight'].shape[1])])]
R_network_params_bias_columns = [*(['bias_'+str(index) for index in range(used_linear_params['bias'].shape[0])])]
R_network_params_weights = pd.DataFrame(used_linear_params['weight'], columns=R_network_params_weight_columns)
R_network_params_bias = pd.DataFrame(used_linear_params['bias'].reshape(1,-1), columns=R_network_params_bias_columns)
save_df_to_csv(path_join(output_path, 'R_network_params_weight.csv'),R_network_params_weights)
save_df_to_csv(path_join(output_path, 'R_network_params_bias.csv'),R_network_params_bias)

In [ ]:
# Sample size
N = 10000

In [ ]:
# Generate two random samples from R_X
A_index = generate_sample_index(data, N, replace=False)
B_index = generate_sample_index_exclude_items(data, N, A_index, replace=False)
R_X_A = R_X.loc[A_index,:].copy(deep=True).reset_index(drop=True)
R_X_B = R_X.loc[B_index,:].copy(deep=True).reset_index(drop=True)
# Save the random samples from R_X
save_df_to_csv(path_join(output_path, 'R_X_A.csv'),R_X_A)
save_df_to_csv(path_join(output_path, 'R_X_B.csv'),R_X_B)

In [ ]:
# You could choose sobolrank (only first order indices), sobolEff (first and total indices) or shapleysobol_knn (first and total indices)
R_sobol_method = 'sobolmultout'
R_sobol_script = path_join(experim_path, R_sobol_method+'_eval.R')

In [ ]:
# Execute the R script (The path to read the data is given as arguments (i.e., experim_path+'\\'))
ouput_R = run([Rscript_path, '--vanilla', R_sobol_script, output_path+'\\', str(data_set_infos['nb_classes'])], shell=True) 

In [ ]:
## Get the important variables per class
# Initialize the determined variables as None
important_variables = None
if R_sobol_method == 'sobolmultout':
    # Load the sobol indices (first and total) and convert it to a dictionary
    R_first_order_sobol_indices = read_csv_to_pd_df(path_join(output_path, R_sobol_method+'_fs'+csv_ext))
    R_total_order_sobol_indices = read_csv_to_pd_df(path_join(output_path, R_sobol_method+'_tt'+csv_ext))
    R_first_order_sobol_dict = R_first_order_sobol_indices['original'].to_dict()
    R_total_order_sobol_dict = R_total_order_sobol_indices['original'].to_dict()
    # Get the important variables per class
    important_variables = important_variables_R_first_and_total_order_analysis_multout_ver(R_first_order_sobol_dict, 
                                                                                           R_total_order_sobol_dict,
                                                                                           filter_method=sobol_filter_method,
                                                                                           divide_factor=4)

In [ ]:
## Build the dataframe that contains the number of neurons
nb_important_vars = len(list(important_variables.keys()))
determined_nb_important_vars = None
if sobol_filter_method != 'top_portion' and sobol_filter_method != 'beside_end_portion':
    determined_nb_important_vars = [[nb_vars, nb_important_vars, sobol_filter_method]]
else:
    determined_nb_important_vars = [[nb_vars, nb_important_vars, sobol_filter_method+'_'+sobol_divide_factor]]
determined_nb_important_vars_df = pd.DataFrame(determined_nb_important_vars, columns=['nb_neurons', 'nb_important_neurons', 'sobol_filter_method'])

In [ ]:
# Build the sorted total important variable(neuron) indices
sorted_important_vars = sorted(list(important_variables))
# Build the mapping dictionary to modify neuron indices
important_var_map_dict = build_map_to_index_dict(important_variables)

In [ ]:
# Save the important variables
store_dict_as_json(path_join(output_path, trained_net_name+'_important_neurons.json'), important_variables)
save_df_to_csv(path_join(output_path, trained_net_name+'_nb_important_neurons.csv'), determined_nb_important_vars_df)

### *Load the distribution shift OOD dataset (Tiny-imagenet-c) and evaluate the activation levels*

In [ ]:
## Read the Tiny-imagnet-c dataset
# Get the content in the folder
transformation_types = [transformation_type for transformation_type in contents_of_folder(tiny_imagenet_c_path)]
# Load all the transformation datasets
load_tiny_imagenet_c_datasets = {}
for transformation_type in transformation_types:
    load_tiny_imagenet_c_datasets[transformation_type] = datasets.ImageFolder(
                                                        root=path_join(tiny_imagenet_c_path, transformation_type),
                                                        transform=v2.Compose([v2.ToTensor()]),
                                                    )

In [ ]:
# Show a Tiny-imagenet-c example
plt.imshow(load_tiny_imagenet_c_datasets['brightness_2'][1300][0].permute(1,2,0), interpolation='nearest')
plt.show()
print(load_tiny_imagenet_c_datasets['brightness_2'][1300][1])

In [ ]:
# Build the dataloader and evaluate the activation levels
distrib_shift_actLevels = {}
for transformation_type in list(load_tiny_imagenet_c_datasets.keys()):
    # Build the loader         
    current_transformed_loader = create_loader_from_torch_dataset(load_tiny_imagenet_c_datasets[transformation_type],
                                                                  batch_size=torch_batch_size, shuffle=False, num_workers=0)
    # Evaluate the activation levels
    distrib_shift_actLevels[transformation_type] = obtain_activation_levels(trained_net, current_transformed_loader,
                                                                            transformation_type, with_predict_class=True,
                                                                            loss_type='cross_entropy')

### *Determine the correctly predicted training examples*

In [ ]:
# Boolean indicating the correctly predicted training examples
correct_train_actLevels = build_correct_actLevels(train_actLevels)

### *Prepare the normalized feature vectors with all neurons*

In [ ]:
# Build the training set normalized feature vectors (only the correctly predicted examples)
correct_train_zs = normalize_feature_vecs_knn(correct_train_actLevels, last_hidden_layerId)

In [ ]:
# Build the training set normalized feature vectors
train_zs = normalize_feature_vecs_knn(train_actLevels, last_hidden_layerId)

In [ ]:
# Build the test set normalized feature vectors
test_zs = normalize_feature_vecs_knn(test_actLevels, last_hidden_layerId)

In [ ]:
# Get the normalized feature vectors of the distribution shift ood set
distrib_shift_zs = {}
for transform_type in distrib_shift_actLevels:
    distrib_shift_zs[transform_type] = normalize_feature_vecs_knn(distrib_shift_actLevels[transform_type], last_hidden_layerId)

### *Prepare the Deep k-nearst neighbors OOD detection with all neurons*

In [ ]:
# Build the faiss train set index
train_index = faiss.IndexFlatL2(correct_train_zs.shape[1])
train_index.add(correct_train_zs)

In [ ]:
# Evaluate the train set k-nearst neighbor scores
correct_train_S = k_nearst_neighbor_scores(train_index, correct_train_zs, k)

In [ ]:
# Get the mean and std of the test scores
threshold_S_mean, threshold_S_std = get_mean_std(correct_train_S)

In [ ]:
# Get the threshold
threshold = threshold_S_mean - std_threshold_coeff * threshold_S_std

### *Deep k-nearst neighbors OOD evaluation result headers*

In [ ]:
knn_ood_eval_headers = ['set_name', 'nb_ood', 'nb_ind', 'ood_percent', 'ind_percent', 'acc_total', 'acc_ood', 'acc_ind']

### *Evaluate the test accuracy after applying the OOD detection with all neurons on the original dataset*

In [ ]:
# Initialize the ood results on the original datasets
origin_ood_eval_results = []
# The ood detection evaluation on the training set
train_ood_eval_result = experim_ood_detection_knn(train_index, train_zs, train_actLevels, k, threshold, 'train')
# Display jump line
print()
# The ood detection evaluation on the test set
test_ood_eval_result = experim_ood_detection_knn(train_index, test_zs, test_actLevels, k, threshold, 'test')
# Add the evaluation results
origin_ood_eval_results.append(train_ood_eval_result)
origin_ood_eval_results.append(test_ood_eval_result)

In [ ]:
# Save the train ood evaluation results
save_list_to_csv(path_join(output_path, 'origin_ood_eval_results'+csv_ext), origin_ood_eval_results, headers=knn_ood_eval_headers)

### *Deep k-nearst neighbors OOD detection on distribution shift dataset (Tiny-imagenet-c)*

In [ ]:
# The ood detection evaluation
distrib_shift_eval_results = []
for transform_type in distrib_shift_zs:
    current_eval_result = experim_ood_detection_knn(train_index, distrib_shift_zs[transform_type],
                                                    distrib_shift_actLevels[transform_type], k, threshold, transform_type)
    distrib_shift_eval_results.append(current_eval_result)
    print()

In [ ]:
# Save the distribution shift ood evaluation results
save_list_to_csv(path_join(output_path, 'distrib_shift_ood_eval_results'+csv_ext), distrib_shift_eval_results, headers=knn_ood_eval_headers)

### *Prepare the normalized feature vectors with only the significant neurons*

In [ ]:
# Take the normalized feature vectors according to the significant neurons of each class for the search index building
correct_train_zs_sig = build_sig_zs(correct_train_actLevels, last_hidden_layerId, sorted_important_vars)

In [ ]:
# Take the normalized feature vectors according to the significant neurons of each class for the training set
train_zs_sig = build_sig_zs(train_actLevels, last_hidden_layerId, sorted_important_vars)

In [ ]:
# Take the normalized feature vectors according to the significant neurons of each class for the test set
test_zs_sig = build_sig_zs(test_actLevels, last_hidden_layerId, sorted_important_vars)

In [ ]:
# Take only the normalized feature vectors from the significant neurons for the distribution shift ood set
distrib_shift_zs_sig = {}
for transform_type in distrib_shift_actLevels:
    distrib_shift_zs_sig[transform_type] = build_sig_zs(distrib_shift_actLevels[transform_type], last_hidden_layerId, sorted_important_vars)

### *Prepare the Deep k-nearst neighbors OOD detection with the significant neurons*

In [ ]:
# Build the faiss train set index
train_index_sig = faiss.IndexFlatL2(correct_train_zs_sig.shape[1])
train_index_sig.add(correct_train_zs_sig)

In [ ]:
# Build the scores over different classes
correct_train_S_sig = k_nearst_neighbor_scores(train_index_sig, correct_train_zs_sig, k)

In [ ]:
# Get the mean and std of the train scores
threshold_S_mean_sig, threshold_S_std_sig = get_mean_std(correct_train_S_sig)

In [ ]:
# Get the threshold
threshold_sig = threshold_S_mean_sig - std_threshold_coeff * threshold_S_std_sig

### *Evaluate the test accuracy after applying the OOD detection with the significant neurons*

In [ ]:
# Initialize the ood results on the original datasets
sig_origin_ood_eval_results = []
# The ood detection evaluation on the training set
sig_train_ood_eval_result = experim_ood_detection_knn(train_index_sig, train_zs_sig, train_actLevels, k, threshold_sig, 'train')
# Display jump line
print()
# The ood detection evaluation on the test set
sig_test_ood_eval_result = experim_ood_detection_knn(train_index_sig, test_zs_sig, test_actLevels, k, threshold_sig, 'test')
# Add the evaluation results
sig_origin_ood_eval_results.append(sig_train_ood_eval_result)
sig_origin_ood_eval_results.append(sig_test_ood_eval_result)

In [ ]:
# Save the train ood evaluation results
save_list_to_csv(path_join(output_path, 'sig_origin_ood_eval_results'+csv_ext), sig_origin_ood_eval_results, headers=knn_ood_eval_headers)

### *Deep k-nearst neighbors OOD detection with only the significant neurons on distribution shift dataset*

In [ ]:
# The ood detection evaluation
sig_distrib_shift_eval_results = []
for transform_type in distrib_shift_zs:
    current_eval_result = experim_ood_detection_knn(train_index_sig, distrib_shift_zs_sig[transform_type],
                                                    distrib_shift_actLevels[transform_type], k, threshold_sig, transform_type)
    sig_distrib_shift_eval_results.append(current_eval_result)
    print()

In [ ]:
# Save the distribution shift ood evaluation results
save_list_to_csv(path_join(output_path, 'sig_distrib_shift_ood_eval_results'+csv_ext), sig_distrib_shift_eval_results, headers=knn_ood_eval_headers)